In [2]:
import uproot3 as up
import mplhep as hep
import matplotlib.pyplot as plt
import numpy as np

from cycler import cycler

hep.style.use('CMS')

In [18]:
config_dict = {
    'dyjetsMC': {'color': '#e5ccff', 'name': 'DY+jets'},
    'hbbMC': {'color': '#ccccff', 'name': r'$H\rightarrow b\bar{b}$'},
    'qcdMC': {'color': '#ffccff', 'name': 'QCD'},
    'vvMC': {'color': '#ffff99', 'name': 'VV'},
    'st': {'color': '#ff9999', 'name': 'Single t'},
    'stMC': {'color': '#ff9999', 'name': 'Single t'},
    'tt': {'color': '#ffcc66', 'name': r'$t\bar{t}$'},
    'ttMC': {'color': '#ffcc66', 'name': r'$t\bar{t}$'},
    'wjets': {'color': '#ccffcc', 'name': 'W+jets'},
    'wjetsMC': {'color': '#ccffcc', 'name': 'W+jets'},
    'zjets': {'color': '#99ffff', 'name': 'Z+jets'},
    'zjetsMC': {'color': '#99ffff', 'name': 'Z+jets'},
}

recoilDict = {
    '0':'[250, 310) GeV',
    '1':'[310, 370) GeV',
    '2':'[370, 470) GeV',
    '3':'[470, 590) GeV',
    '4':'[590, inf] GeV'
}

regionDict = {
    'sr_pass':'Signal Region',
    'sr_fail':'Z+jets Control Region',
    'wecr_pass':'W+jets Single Electron \"Pass\" Control Region',
    'wecr_fail':'W+jets Single Electron \"Fail\" Control Region',
    'wmcr_pass':'W+jets Single Muon \"Pass\" Control Region',
    'wmcr_fail':'W+jets Single Muon \"Fail\" Control Region',
    'tecr_pass':'Top-Pair Single Electron Control Region',
    'tmcr_pass':'Top-Pair Single Muon Control Region',
}

In [12]:
def merge_data(input_file):

    years = ['2016', '2017', '2018']
    region = 'sr'
    recoil_tags = [str(i) for i in range(5)]
    categories = ['pass', 'fail']

    outputs = {}

    for itag in recoil_tags:
        for category in categories:

            merged_data = None

            for year in years:
                identifier40to120 = f'{region}{year}{category}mass40to120recoil{itag}'
                identifier120to300 = f'{region}{year}{category}mass120to300recoil{itag}'

                prefit_dir40to120 = input_file[identifier40to120+'_prefit']
                prefit_dir120to300 = input_file[identifier120to300+'_prefit']

                data40to120 = prefit_dir40to120["data_obs"].values
                data120to300 = prefit_dir120to300["data_obs"].values

                data = np.concatenate((data40to120, data120to300), axis=None)

                if merged_data is None:
                    merged_data = np.array(data)  # Initialize as a NumPy array
                else:
                    merged_data += data  # Element-wise addition

            outputs[f'{category}_{itag}'] = merged_data

    return outputs


def merge_bkg(input_file, processes40to120, processes120to300):

    years = ['2016', '2017', '2018']
    region = 'sr'
    recoil_tags = [str(i) for i in range(5)]
    categories = ['pass', 'fail']

    edges = [40.,  50.,  60.,  70.,  80.,  90., 100., 120., 150., 180., 240., 300.]

    outputs = {}

    for itag in recoil_tags:
        for category in categories:

            sum_prefit = np.zeros(len(edges)-1)
            sum_postfit = np.zeros(len(edges)-1)
            sum_postfit_variance = np.zeros(len(edges)-1)

            for year in years:
                identifier40to120 = f'{region}{year}{category}mass40to120recoil{itag}'
                identifier120to300 = f'{region}{year}{category}mass120to300recoil{itag}'

                prefit_dir40to120 = input_file[identifier40to120+'_prefit']
                prefit_dir120to300 = input_file[identifier120to300+'_prefit']

                postfit_dir40to120 = input_file[identifier40to120+'_postfit']
                postfit_dir120to300 = input_file[identifier120to300+'_postfit']

                for (i, j) in zip(processes40to120, processes120to300):

                    try:
                        prefit40to120 = prefit_dir40to120[i].values
                        postfit40to120 = postfit_dir40to120[i].values
                        postfit40to120_variance = np.minimum(postfit40to120, postfit_dir40to120[i].variances)
                    except:
                        continue

                    try:
                        prefit120to300 = prefit_dir120to300[j].values
                        postfit120to300 = postfit_dir120to300[j].values
                        postfit120to300_variance = np.minimum(postfit120to300, postfit_dir120to300[j].variances)
                    except:
                        continue

                    prefit_bin = np.concatenate((prefit40to120, prefit120to300), axis=None)
                    postfit_bin = np.concatenate((postfit40to120, postfit120to300), axis=None)
                    postfit_bin_variance = np.concatenate((postfit40to120_variance, postfit120to300_variance), axis=None)
                    sum_prefit += prefit_bin
                    sum_postfit += postfit_bin
                    sum_postfit_variance += postfit_bin_variance

            outputs[f'{category}_{itag}_prefit'] = sum_prefit
            outputs[f'{category}_{itag}_postfit'] = sum_postfit
            outputs[f'{category}_{itag}_postfit_variance'] = sum_postfit_variance

    return outputs

def merge_individual_bkg(input_file, processes40to120, processes120to300):

    from collections import defaultdict

    years = ['2016', '2017', '2018']
    region = 'sr'
    recoil_tags = [str(i) for i in range(5)]
    categories = ['pass', 'fail']

    process_bin = {}
    process_bin_variance = {}

    for itag in recoil_tags:
        for category in categories:

            for year in years:
                identifier40to120 = f'{region}{year}{category}mass40to120recoil{itag}'
                identifier120to300 = f'{region}{year}{category}mass120to300recoil{itag}'

                postfit_dir40to120 = input_file[identifier40to120+'_postfit']
                postfit_dir120to300 = input_file[identifier120to300+'_postfit']

                for (i, j) in zip(processes40to120, processes120to300):

                    try:
                        postfit40to120 = postfit_dir40to120[i].values
                        postfit40to120_variance = np.minimum(postfit40to120, postfit_dir40to120[i].variances)
                    except:
                        continue

                    try:
                        postfit120to300 = postfit_dir120to300[j].values
                        postfit120to300_variance = np.minimum(postfit120to300, postfit_dir120to300[j].variances)
                    except:
                        continue

                    postfit_bin = np.concatenate((postfit40to120, postfit120to300), axis=None)
                    postfit_bin_variance = np.concatenate((postfit40to120_variance, postfit120to300_variance), axis=None)

                    key = f'{itag}_{category}_{i}_{j}'

                    if not key in process_bin.keys():
                        process_bin[key] = postfit_bin
                        process_bin_variance[key] = postfit_bin_variance
                    else:
                        process_bin[key] += postfit_bin
                        process_bin_variance[key] += postfit_bin_variance

    return process_bin, process_bin_variance

In [ ]:
### Load input file
f = up.open("../hists/darkhiggs.postfit")

### Merged data
data_values_dict = merge_data(f)

### Merged MC
processes40to120 = ['hbbMC', 'dyjetsMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'ttMC', 'wjets', 'wjetsMC', 'zjets', 'zjetsMC']
processes120to300 = ['hbbMC', 'dyjetsMC', 'qcdMC', 'vvMC', 'stMC', 'tt',  'ttMC', 'wjets', 'wjetsMC', 'zjets', 'zjetsMC']

total_bkg_dict = merge_bkg(f, processes40to120, processes120to300)
indiv_mc, indiv_mc_variance = merge_individual_bkg(f, processes40to120, processes120to300)

In [ ]:
data_values_dict.keys()

In [ ]:
total_bkg_dict.keys()

In [ ]:
indiv_mc.keys()

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(40,12), sharex='col', sharey='row',
                                gridspec_kw=dict(height_ratios=[3, 1], hspace=0.15, wspace=0.))
errps = {'hatch':'////', 'facecolor':'none', 'lw': 0, 'color': 'k', 'alpha': 0.3}
edges = [40.,  50.,  60.,  70.,  80.,  90., 100., 120., 150., 180., 240., 300.]

fig.supxlabel('AK15 Soft-Drop Mass [GeV]', fontsize=40)

for k in range(5):

    ax=axs[0][k]
    rax=axs[1][k]

    ax.text(s=f'$U$ $\in$ {recoilDict[str(k)]}',x=0.02,y=0.92, fontsize=35,transform=ax.transAxes)
    ax._get_lines.prop_cycler = ax._get_patches_for_fill.prop_cycler
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e+5)

    colors=[]
    mc_process_data = []
    mc_names = []
    for mc in indiv_mc.keys():
        if mc.split('_')[0] != str(k):
            continue
        elif mc.split('_')[1] == 'fail':
            continue
        colors.append(config_dict[mc.split('_')[-1]]['color'])
        mc_process_data.append(indiv_mc[mc])
        mc_names.append(config_dict[mc.split('_')[-1]]['name'])

    ax.set_prop_cycle(cycler(color=colors))

    ### Try to draw stack plots
    hep.histplot(mc_process_data, edges, ax=ax, stack=True,
                    histtype='fill', edgecolor = 'k', linewidth=3, label=mc_names)

    ### Draw stat uncs.
    y1 = total_bkg_dict[f'pass_{k}_postfit'] - np.sqrt(total_bkg_dict[f'pass_{k}_postfit_variance'])
    y1 = np.append(y1, 0)
    y2 = total_bkg_dict[f'pass_{k}_postfit'] + np.sqrt(total_bkg_dict[f'pass_{k}_postfit_variance'])
    y2 = np.append(y2, 0)
    ax.fill_between(
        x = edges,
        y1 = y1,
        y2 = y2,
        step = 'post',
        **errps, label='Stat. Unc.'
    )

    hep.histplot(total_bkg_dict[f'pass_{k}_postfit'], edges, ax=ax, label=["Postift"], color='b', linewidth=6)
    hep.histplot(total_bkg_dict[f'pass_{k}_prefit'], edges, ax=ax, label=["Prefit"], color='r',
                    linestyle='dashed', linewidth=6)

    ### Call data ###
    hep.histplot(data_values_dict[f'pass_{k}'], edges, ax=ax, histtype='errorbar', label="Data", color='k', markersize=20)

    #### Draw ratio ####
    hep.histplot(data_values_dict[f'pass_{k}']/total_bkg_dict[f'pass_{k}_prefit'], edges, yerr=np.sqrt(data_values_dict[f'pass_{k}'])/total_bkg_dict[f'pass_{k}_prefit'], ax=rax, histtype='errorbar',
                     color='r', capsize=10, label="Prefit", markersize=20)
    hep.histplot(data_values_dict[f'pass_{k}']/total_bkg_dict[f'pass_{k}_postfit'], edges, yerr=np.sqrt(data_values_dict[f'pass_{k}'])/total_bkg_dict[f'pass_{k}_postfit'], ax=rax, histtype='errorbar',
                    color='b', capsize=10, label="Postfit", markersize=20)

    y1 = 1.- np.sqrt(total_bkg_dict[f'pass_{k}_postfit'])/total_bkg_dict[f'pass_{k}_postfit']
    y1 = np.append(y1, 0)
    y2 = 1.+ np.sqrt(total_bkg_dict[f'pass_{k}_postfit'])/total_bkg_dict[f'pass_{k}_postfit']
    y2 = np.append(y2, 0)

    rax.fill_between(
        x = edges,
        y1 = y1,
        y2 = y2,
        step='post',
        **errps, label='Stat. Uncs.'
    )

    rax.axhline(1, ls='--', color='k')
    ymax=abs(rax.get_ylim()[1])*1.1
    ymin=1.-(ymax-1.)
    rax.set_ylim(max(ymin,0),min(ymax,2))
    rax.set_xlim(40, 300)

    if k == 0:
        hep.cms.text(ax=ax, loc=0, text='Preliminary',fontsize=45)
        ax.set_ylabel('Events/GeV', fontsize=40)
        ax.tick_params(axis='y', labelsize=30)
        ax.set_xticks([50,100,150,200,250])
        rax.set_ylabel('Obs/Exp', fontsize=40)
        rax.tick_params(axis='y', labelsize=30)
        rax.tick_params(axis='x', labelsize=30)
        handles, labels = ax.get_legend_handles_labels()

    else:
        ax.set_xticks([50,100,150,200,250])
        rax.tick_params(axis='x', labelsize=30)

    if k == 4:

        ### Lumi text
        hep.cms.lumitext(ax=ax, text=r"138 fb$^{-1}$, 2016-2018 (13 TeV)", fontsize=45)

        #### Legend
        order = [11, 9, 10, 7, 6, 5, 4, 3, 2, 1, 0, 8]
        ax.legend([handles[idx] for idx in order], [labels[idx] for idx in order],
                     loc='center left', fontsize=20, ncol=3, bbox_to_anchor=(-0.02, 0.72))

        handles, labels = rax.get_legend_handles_labels()
        order = [1, 2, 0]
        rax.legend([handles[idx] for idx in order], [labels[idx] for idx in order],
                   loc='center left', fontsize=20, ncol=3, bbox_to_anchor=(0.0, 0.12))